In [ ]:
import json
import random

#Function that loads the game data from a JSON file
def load_game_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data['gamebjects'], data['playerClasses'], data['enemies'], data['bosses']

#PLAYER CLASS
#Initializes player class or whatever
#____________________________________________________________________________________________________________________________________________________
class Player:
    def __init__(self, name, player_class_data, game_objects):
        self.name = name
        self.player_class = player_class_data['name']
        self.max_hp = player_class_data['max_hp']
        self.hp = self.max_hp
        self.max_sp = playerclass_data['max_sp']
        self.sp = self.max_sp
        self.special_attacks = player_class_data.get('special_attacks', [])
        # Initialize inventory
        self.inventory = {item['name']: {'effect': item['effect'], 'quantity': item.get('quantity', 1)} for item in game_objects}
        
#INVENTORY   
#Displays current inventory items with quantity and effects
    def display_inventory(self):
        print("Inventory:")
        for idx, (item_name, item_details) in enumerate(self.inventory.items(), start=1):
            print(f"{idx}. {item_name}: {item_details['quantity']} - {item_details['effect']}")
        
    #Attack Functionality
    def attack(self, enemy):
        print(f"{self.name} attacks the enemy!")
        enemy.hp -= random.randint(10, 40)
        print(f"Enemy HP: {enemy.hp}")
    
    #Special Attack Functionality
    def special_attack(self, enemy, attack_idx):
        if attack_idx < len(self.special_attacks):
            special_attack = self.special_attacks[attack_idx]
            if self.sp >= special_attack['sp_cost']:
                print(f"{self.name} uses {special_attack['name']}!")
                enemy.hp -= special_attack['damage']
                self.sp -= special_attack['sp_cost']
                print(f"Enemy HP: {enemy.hp}")
            else:
                print("Not enough special points!")
        else:
            print("Invalid special attack index!")

    #Item Functionality
    def use_item(self, item_choice, enemy=None):  # Now accepts enemy as an optional argument
        item_keys = list(self.inventory.keys())
        if item_choice.isdigit():
            item_index = intitem_choice) - 1
            if 0 <= item_index < len(item_keys):
                item_name = item_keys[item_index]
                item_data = self.inventory[item_name]
                if item_data['quantity'] > 0:
                    # Now also passes enemy to apply_item_effect
                    self.apply_item_effect(item_name, item_data, enemy)
                    if item_data['quantity'] == 0:
                        print(f"{item_name} has been fully used.")
                else:
                    print(f"No more {item_name} left.")
            else:
                print("Invalid item selection.")
        else:
            print("Please enter a valid number.")

    #Apply Item Effect Functionality
    def apply_item_effect(self, item_name, item_data, enemy):  # Now properly accepts enemy
        if "Restore" in item_data['effect']:
            self.restore_effect(item_data['effect'], item_name)
        elif "Deal" in item_data['effect']:
            # Now correctly checks for enemy before trying to deal damage
            if enemy:
                self.deal_effect(item_data['effect'], item_name, enemy)
            else:
                print(f"Cannot use {item_name} outside of combat.")
        item_data['quantity'] -= 1

    #Helps for Healing SP and HP with Item
    def restore_effect(self, effct, item_name):
        parts = effect.split()
    # Assuming the format "Restore 20 HP" or "Restore 15 SP"
        amount = int(parts[1])  # The second part is the amount
        stat = parts[2]  # The third part is the stat type (HP or SP)
        if stat == "HP":
            healed_amount = min(self.hp + amount, self.max_hp) - self.hp
            self.hp += healed_amount
            print(f"{self.name} uses {item_name}. {healed_amount} HP restored to {self.hp}/{self.max_hp}.")
        elif stat == "SP":
            restored_amount = min(self.sp + amount, self.max_sp) - self.sp
            self.sp += restored_amount
            print(f"{self.name} uses {item_name}. {restored_amount} SP restored to {self.sp}/{self.max_sp}.")

    #deals damage with Item (EMOTIONAL DAMAGE)
    def deal_effect(self, effect, item_name, enemy):
        parts = effect.split()
        damage = int(parts[1])  # Assuming the second word is always the damage amount
        enemy.hp -= damage
        print(f"{self.name} uses {item_name}. {damage} damage dealt to the enemy.")
        print(f"Enemy HP: {enemy.hp}")
        
#END OF PLAYER CLASS 
#______________________________________________________________________________________________________________________________________________

#ENEMY CLASS
#Initializes Enemy class (boooooo)
#__________________________________________________________________________________________________________________________________________________
class Enemy:
    def __init__(self, name, max_hp):
        self.max_hp = max_hp
        self.hp = self.max_hp

    def attack(self, player):
        damage = random.randint(10, 30)
        player.hp -= damage
        print(f"{selname} attacks {player.name} for {damage} damage. {player.name} HP: {player.hp}")

    def special_attack(self, player):
        damage = random.randint(20, 40)
        player.hp -= damage
        print(f"{self.name} uses a special attack on {player.name} for {damage} damage. {player.name} HP: {player.hp}")
        
#END OF ENEMY CLASS
#______________________________________________________________________________________________________________________________________________     

#GAME LOOP
#Initial Game Functionality is here, loops until you go kaput
#_____________________________________________________________________________________________________________________________________________________________
def game_loop(player, enemies_data, bosses_data, game_objects):
    enemy_count = 0
    while player.hp > 0:
        # Determine whether it's time for a boss battle
        if enemy_count % 4 == 0 and ene_count != 0:
            enemy_data = random.choice(bosses_data)
            print("A mighty boss appears!")
        else:
            enemy_data = random.choice(enemies_data)

        hp_range = enemy_data['hp_range'].split('-')
        enemy_hp = random.randint(int(hp_range[0]), int(hp_range[1]))    
            
        enemy = Enemy(enemy_data['name'], enemy_hp)
        print(f"\n***************Battle Start!***************")
        print(f"You encounter a {enemy.name} with {enemy.hp} HP!")
        
        while player.hp > 0 and enemy.hp > 0:
            print(f"\nYour HP: {player.hp}/{player.max_hp}, Your SP: {player.sp}/{player.max_sp}")  # Displaying player's current HP and SP
            print("\nPlayer's Turn:")
            print("1. Attack")
            print("2. Special Attack")
            print("3. Use Item")
            choice = input("Choose your action: \n")

            if choice == '1':
                player.attack(enemy)
            
            elif choice == '2':
                print("Available Special Attacks:")
                for idx, special_attack in enumerate(player.special_attacks):
                    print(f"{idx + 1}. {special_attack['name']} - {special_attack['description']} (Cost: {special_attack['sp_cost']} SP)")
                special_attack_choice = int("Choose the special attack to use: ")
                player.special_attack(enemy, int(special_attack_choice) - 1)
            
            elif choice == '3':
                player.display_inventory()  # Display the inventory with detailed information
                item_choice = input("Choose an item to use: ").strip()
                if item_choice.isdigit():
                    item_index = int(item_choice) - 1
                    item_keys = list(player.inventory.keys())
                    if 0 <= item_index < len(item_keys):
                        player.use_item(item_choice, enemy)  # Now correctly passing the item choice and enemy
                    else:
                        print("Invalid selection. Please choose a valid item number.")
                else:
                    print("Please enter a number corresponding to the item you wish to use.")

            if enemy.hp <= 0:
                print("Enemy defeated!")
                enemy_count += 1
                break

            print("\nEnemy's Turn:")
            enemy_choice = random.choice(['1', '2'])
            if enemy_choice == '1':
                enemy.attack(player)
            elif enemy_choice == '2':
                enemy.special_attack(player)

        if player.hp <= 0:
            print("You have been defeated!")
            break    
            
#END OF GAME LOOP SECTION
#_____________________________________________________________________________________________________________________________

#MAIN Function that starts the game and Navigation through menu is here
#_____________________________________________________________________________________________________________________________________________
def main():
    game_objects, player_classes, enemies_data, bosses_data = load_game_data('GameObjectsList.json')
    player_name = input("Enter your name: ")
    player_class_data = player_classes[0]  # Assuming selection is made here
    player = Player(player_name, player_class_data, game_objects)

    print(f"\nWelcome, {player.name}, to the battle!")
    game_loop(player, enemies_data, bosses_data, game_objects)
        
#_________________________________________________________________________________________________________________________________________________________________
#Main Menu
#_________________________________________________________________________________________________________________________________________________________
def main_menu():
    print("Welcome to the One Piece RPG Game!")
    print("1. Start Game")
    print("2. Instructions")
    print("3. Exit")
    while True:
        choice = input("Enter your choice: ")
        if choice == '1':
            main()
            break  # Break after main to return to menu after game over.
        elif choice == '2':
            print_instructions()
        elif choice == '3':
            print("Exiting game. Goodbye!")
            break  # Exit the loop to end the program.
        else:
            print("Not a valid entry. Please enter a number between 1 and 3.")

#Instruction Functionality
def print_instructions():
    print("\n*** Game Instructions ***")
    print("1. Start the game by choosing your class. Each class has unique abilities.")
    print("2. You will encounter enemies. Choose what attack you want to use. Use your items and SP wisely.")
    print("3. Use 'Attack' to deal damage, 'Special Attack' to use class-specific moves, and 'Use Item' to heal or deal extra damage.")
    print("4. Keep an eye on your HP and SP. If your HP drops to 0, you lose. Items give you another means of doing damage if all else runs low.")
    print("5. You can carry and use items. Manage your inventory efficiently.")
    print("6. Defeating enemies may reward you with more items. You will keep fighting until you can no longer fight. ")
    print("\nGood luck on your battle adventure!\n")
    input("Press Enter to return to the main menu...")
    main_menu()

#Main entry point of program
if __name__ == "__main__":
    main_menu()

Welcome to the One Piece RPG Game!
1. Start Game
2. Instructions
3. Exit
